In [ ]:
%load_ext autoreload
%autoreload 2
# For reloading code such that code changes are taken into account

In [ ]:
project_id = 'test-project-221821'


import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join(os.environ['HOME'], 'gcp/secret/admin-service-account.json')
import sys
sys.path.append(os.path.join(os.environ['HOME'], 'gcp'))

from google.cloud import bigquery
from bigquery.client import Bigquery

import pytz
import names
import numpy as np
import datetime as dt

# Create a dataset with tables

In [ ]:
dataset_id = 'test_dataset'
location = 'EU'
bq = Bigquery(project_id, location, dataset_id)

In [ ]:
bq.create_dataset()

In [ ]:
# table 1
table_users = 'users'
schema_users = [bigquery.SchemaField('full_name', 'STRING', mode='REQUIRED'),
                bigquery.SchemaField('age', 'INTEGER', mode='REQUIRED')]
bq.create_table(table_users, schema_users)

# table 2
table_purchases = 'purchases'
schema_purchases = [bigquery.SchemaField('purchase_time', 'TIMESTAMP', mode='REQUIRED'),
                    bigquery.SchemaField('full_name', 'STRING', mode='REQUIRED'),
                    bigquery.SchemaField('products', 'STRING', mode='REPEATED')]
bq.create_table(table_purchases, schema_purchases)

# Generate some random data and insert

In [ ]:
users_data = [[names.get_first_name(), np.random.randint(18, 60)] for i in range(10)]

In [ ]:
insert_result = bq.streaming_insert(users_data, table_users)

In [ ]:
purchases_data = [(dt.datetime(2018, 1, np.random.randint(1, 32), np.random.randint(0, 24), tzinfo=pytz.UTC),
                  names.get_first_name(),
                  ['product{}'.format(j) for j in range(np.random.randint(1,4))]) for i in range(10)]

In [ ]:
insert_result = bq.streaming_insert(purchases_data, table_purchases)

# Create a query and execute

In [ ]:
query = "SELECT * FROM `{}.users` WHERE age > 9".format(bq.dataset_id)
# it is possible to validate your query first to prevent making unnecessary costs
dry_run_result = bq.dry_run_query(query)
dry_run_result

In [ ]:
result = bq.execute_query(query)
result = list(result)
result

In [ ]:
# Doing an invalid query
bq.execute_query("SELECT * from henk", validate_query=True)

# Delete the dataset with all its tables

In [ ]:
bq.delete_dataset()